# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/content/1_bank_client.csv')
print(df.head(3))
print('\n\n')
print(df.info())
print('\n\n')
print(df.isna().sum())

   children  days_employed  ...   total_income                  purpose
0         1   -8437.673028  ...  253875.639453            покупка жилья
1         1   -4024.803754  ...  112080.014102  приобретение автомобиля
2         0   -5623.422610  ...  145885.952297            покупка жилья

[3 rows x 12 columns]



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_in

### Вывод

В датасете наблюдается 12 столбцов и 21525 строк. В данных присутствуют ни коректные наблюдении в стольбце "Стаж работы" - отрицательные значения, так ж в стольбце 'days_employed' и 'total_income' пропущенные значения, причём они находятся в одних и тех же строках.

In [3]:
# найдем среднее и медианное значение в столбцах 'days_employed' и 'total_income'
median_days_employed = df['total_income'].median()
mean_days_employed = df['days_employed'].mean()

# заменим отрицательные числа на положительные в стаже
def negativ_to_positive(value):
  if value < 0:
    value *= -1
    return value
  else:
    return value
df['days_employed'] = df['days_employed'].apply(negativ_to_positive)

# заменяем пропущенные значения    
df['days_employed'] = df['days_employed'].fillna(mean_days_employed)
df['total_income'] = df['total_income'].fillna(median_days_employed)

> В стаже отрицательные числа сделали положительными, пропуски заменили на среднее значение. В доходе пропуски заменили на медианное значение.
Обратили внимание на неадекватность стажа у пенсионеров, стоят завышенные значения, в 25% выборки. При использовании этих значений в расчетах, мы бы получили неправльную картину. О причинах неадекватности значений надо было бы уточнить у операторов и исправить ошибку.

In [4]:
# приводим к значения в колонках к целочисленным
df['days_employed'] = df['days_employed'].astype(int)
df['total_income'] = df['total_income'].astype(int)

In [5]:
# категориальные столбцы
categor_colum = list(df.select_dtypes(include=['object']).columns)

# численные столбцы
num_colum = [x for x in df.columns if x not in categor_colum] 

# стандартизация строк
for row in categor_colum:
  df[row].str.lower

df.duplicated().sum()

54

In [6]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

In [14]:
#проверим формат столбца количества детей
df['children'].value_counts()
#видим ошибку - присутствует значение -1, а также 20 в столбце количества детей
# -1 заменим на 1, а 20 заменим медианным значением, так как 
# мы не знаем как возникла ошибка: случайно ввели лишний ноль при вводе 2 или же вместо 0 случайно поставили 20
df['children'] = df['children'].replace(-1, 1)
children_median = df.loc[df.loc[:, 'children'] != 20]['children'].median()
df['children'] = df['children'].replace(20, children_median)
pd.DataFrame({'Кол-во заёмщиков по кол-ву детей':df['children'].value_counts()})

,Кол-во заёмщиков по кол-ву детей
0,14183
1,4856
2,2052
3,330
4,41
5,9


## Лемматизация 

In [13]:
#столбец цель кредита нужно лемматизировать и на основе его создать новый столбец с категориями 
from collections import Counter
estate = ['недвижим', 'жиль']
wedding = ['свадьб']
auto = ['автомоб']
education = ['образова']

df['purpose_category'] = 0
def categoryzer(list_of_words, category):
  join = '|'.join(list_of_words)
  index = df[df['purpose'].str.lower().str.contains(join)].index.to_list()
  for i in index:
    df.loc[i, 'purpose_category'] = category
  return df


categoryzer(wedding, 'свадьба')
categoryzer(estate, 'недвижимость')
categoryzer(auto, 'авто')
categoryzer(education, 'образование')

#проверим результат и убедимся, что значений None не осталось
Counter(df['purpose_category'])

Counter({'авто': 4308,
         'недвижимость': 10814,
         'образование': 4014,
         'свадьба': 2335})

>Видим, что многие уникальные значения цели кредита названы похожим образом, но имеют одинаковый смысл. Мы лемматизируем их, чтобы затем разбить на категории. Обе задачи мы выполнили с помощью одной функции. В итоге мы получили 4 категории целей кредита. В нашей задаче это не понадобится, но наша категоризация сможет помочь другим аналитикам, которые изучают влияние цели кредита на возврат денег в срок.

In [15]:
debt_from_children = pd.DataFrame()
debt_from_children['count_children'] = df.groupby('children')['debt'].count()
debt_from_children['sum_children'] = df.groupby('children')['debt'].sum()
debt_from_children['result_children'] = debt_from_children['sum_children'] / debt_from_children['count_children'] 
debt_from_children

,count_children,sum_children,result_children
children,,,
0,14183,1071,0.075513
1,4856,445,0.091639
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


> Так как мы изучаем влияние семейного положения и наличия детей на факт погашения в срок, мы не будем разбивать возраста на категории, так как они нам не понадобятся. Для сдачи проекта категоризация была применена вместе с лемматизацией выше в одной функции. Категоризовать количество детей нет необходимости, категориями будет как раз их количество, так как всего 6 вариантов.

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [16]:
debt_from_children = pd.DataFrame()
debt_from_children['count_children'] = df.groupby('children')['debt'].count()
debt_from_children['sum_children'] = df.groupby('children')['debt'].sum()
debt_from_children['result_children'] = debt_from_children['sum_children'] / debt_from_children['count_children'] 
debt_from_children.sort_values('result_children', ascending = False)

,count_children,sum_children,result_children
children,,,
4,41,4,0.097561
2,2052,194,0.094542
1,4856,445,0.091639
3,330,27,0.081818
0,14183,1071,0.075513
5,9,0,0.000000


> С увеличением количества детей мы видим увеличение количества просроченных задолженностей, хотя люди с 3 детьми чаще платят в срок чем люди с 1 ребенком. Данные не однозначные, возможно, нужна большая выборка, чем та, которую мы имеем. 
Бездетные, как правило реже просрачивают оплату по кредиту, чем люди с детьми.

### Есть ли зависимость между семейным статусом и возвратом кредита в срок?

In [17]:
debt_from_family_status = pd.DataFrame()
debt_from_family_status['sum_family_status'] = df.groupby('family_status')['debt'].sum()
debt_from_family_status['count_family_status'] = df.groupby('family_status')['debt'].count()
debt_from_family_status['result_family_status'] = debt_from_family_status['sum_family_status'] / debt_from_family_status['count_family_status'] 
debt_from_family_status.sort_values('result_family_status', ascending = False)

,sum_family_status,count_family_status,result_family_status
family_status,,,
Не женат / не замужем,274,2810,0.097509
гражданский брак,388,4163,0.093202
женат / замужем,931,12344,0.075421
в разводе,85,1195,0.071130
вдовец / вдова,63,959,0.065693


> Да, зависимость есть. Люди не в браке и не бывавшие в браке имеют больший процент невозвратов в срок. 
Но, те кто развелись или овдовели чаще платят в срок, чем люди в браке.

### Как разные цели кредита влияют на его возврат в срок?

In [18]:
debt_from_purpose_category = pd.DataFrame()
debt_from_purpose_category['sum_purpose_category'] = df.groupby('purpose_category')['debt'].sum()
debt_from_purpose_category['count_purpose_category'] = df.groupby('purpose_category')['debt'].count()
debt_from_purpose_category['result_purpose_category'] = debt_from_purpose_category['sum_purpose_category'] / debt_from_purpose_category['count_purpose_category'] 
debt_from_purpose_category.sort_values('result_purpose_category', ascending = False)

,sum_purpose_category,count_purpose_category,result_purpose_category
purpose_category,,,
авто,403,4308,0.093547
образование,370,4014,0.092177
свадьба,186,2335,0.079657
недвижимость,782,10814,0.072314


> Как мы видим, среди 4 наших категорий, чаще всего платят в срок с целью кредита недвижимость, чаще просрочка у тех кому нужен кредит на авто и образование. Кредиты на свадьбу просрачивают реже - примерно как и в случае недвижимости.

## Общий вывод

Общий вывод - семейный статус влияет на вероятность платежей по кредиту в срок - люди, бывший в браке чаще плятят в срок чем те, кто не бывали в браке, причем разведенные и овдовевшие платят в срок чаще, чем люди в браке. Чем больше детей, тем чаще платят не в срок, бездетные реже просрачивают оплату.